### Basic setup

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.model_selection import train_test_split
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import json
import os
import tensorflow as tf

In [ ]:
ROOT_DIR = '/kaggle/input/cassava-leaf-disease-classification/'
BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
train = pd.read_csv(ROOT_DIR + 'train.csv')
sample_submission = pd.read_csv(ROOT_DIR + 'sample_submission.csv')

In [ ]:
print('Train size: ', train.shape)

In [ ]:
train.head()

In [ ]:
with open('/kaggle/input/cassava-leaf-disease-classification/label_num_to_disease_map.json', 'r') as f:
    diseases = json.load(f)
print(diseases)

### Visualize pictures for each disease

For more details about each disease, you can refer to this discussion: https://www.kaggle.com/c/cassava-leaf-disease-classification/discussion/198143

#### Cassava Bacterial Blight

In [ ]:
def visualize_disease(disease):
    fig = plt.figure(figsize=(15, 10))
    columns = 2
    rows = 2
    imgnames = list(train[train['label']==disease].iloc[:4]['image_id'])
    for i in range(len(imgnames)):
        img_name = imgnames[i]
        im = cv2.imread(ROOT_DIR + 'train_images/' + img_name)
        fig.add_subplot(rows, columns, int(i)+1)
        plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        plt.title(img_name)

In [ ]:
visualize_disease(0)

#### Cassava Brown Streak Disease

In [ ]:
visualize_disease(1)

#### Cassava Green Mottle

In [ ]:
visualize_disease(2)

#### Cassava Mosaic Disease

In [ ]:
visualize_disease(3)

#### Healthy

In [ ]:
visualize_disease(4)

### Check disease distribution

In [ ]:
sns.countplot(x='label', data=train)

Check image shape

In [ ]:
# image_shapes = [cv2.imread(ROOT_DIR + 'train_images/' + img_name).shape for img_name in train['image_id']]

In [ ]:
# np.unique(image_shapes)

So all the images have the same shape 600x800x3

#### Inspect Train TFRecord files

In [ ]:
train_filenames = tf.io.gfile.glob(ROOT_DIR + 'train_tfrecords/' + 'ld_train*.tfrec')
train_set = tf.data.TFRecordDataset(train_filenames) 
for raw_record in train_set.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(dict(example.features.feature).keys())

#### Inspect Test TFRecord files

In [ ]:
test_filenames = tf.io.gfile.glob(ROOT_DIR + 'test_tfrecords/' + 'ld_test*.tfrec')
test_set = tf.data.TFRecordDataset(test_filenames) 
for raw_record in test_set.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(dict(example.features.feature).keys())

#### Split into train, validation set

In [ ]:
train_filenames, val_filenames = train_test_split(train_filenames, test_size=0.3, random_state=37)

In [ ]:
def augment(image, label):
    if tf.random.uniform([1], dtype='float32') < 0.5:
        image = tf.image.resize_with_crop_or_pad(image, 224+6, 224+6)
        image = tf.image.random_crop(image, size=[224, 224, 3])
        image = tf.image.central_crop(image, 0.6)
        image = tf.image.resize(image, (224, 224))
    return image, label

def _parse_function(example, feature_description):
    parsed_example = tf.io.parse_single_example(example, feature_description)
    image = tf.io.decode_jpeg(parsed_example['image'], channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.resnet50.preprocess_input(image)
    if 'target' in feature_description:
        target = tf.cast(parsed_example['target'], tf.int32)
        return image, target
    return image, parsed_example['image_name']


def load_data(filenames, ordered, labeled):
    options = tf.data.Options()
    if not ordered:
        options.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE)
    dataset = dataset.with_options(options)
    if labeled:
        feature_description = {
            'image': tf.io.FixedLenFeature([], tf.string),
            'target': tf.io.FixedLenFeature([], tf.int64)
        }
    else:
        feature_description = {
            'image': tf.io.FixedLenFeature([], tf.string),
            'image_name': tf.io.FixedLenFeature([], tf.string)
        }
    parsed_dataset = dataset.map(lambda x: _parse_function(x, feature_description), num_parallel_calls=AUTOTUNE)
    return parsed_dataset

def get_train_set(filenames, batch_size=32):
    dataset = load_data(filenames, ordered=False, labeled=True)
    dataset = dataset.map(lambda x, y: augment(x, y), num_parallel_calls=AUTOTUNE)
    dataset = dataset.shuffle(37)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

def get_val_set(filenames, batch_size=32):
    dataset = load_data(filenames, ordered=True, labeled=True)
    dataset = dataset.batch(batch_size)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

def get_test_set(filenames, batch_size=32):
    dataset = load_data(filenames, ordered=True, labeled=False)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
train_set = get_train_set(train_filenames, batch_size=BATCH_SIZE)
val_set = get_val_set(val_filenames, batch_size=BATCH_SIZE)
test_set = get_test_set(test_filenames, batch_size=BATCH_SIZE)

### Modelling

In [ ]:
base_model = tf.keras.applications.ResNet50(include_top=False, input_shape=(224,224,3), weights='imagenet')
base_model.trainable = False # freeze pretrained model's weights
augmentation_layer = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomFlip(),
])
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dropout_layer = tf.keras.layers.Dropout(0.3)
prediction_layer = tf.keras.layers.Dense(5, activation='softmax')

# create model based on TF's Functional API
inputs = tf.keras.Input(shape=(224, 224, 3))
x = augmentation_layer(inputs)
x = base_model(x, training=False) # remember to set training=False
x = global_average_layer(x)
x = dropout_layer(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4)
model.compile(optimizer=optimizer, 
              loss='sparse_categorical_crossentropy', 
              metrics=['sparse_categorical_accuracy'])

In [ ]:
initial_epochs = 15
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-5, verbose=1)
early_stopping_cb =  tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True, verbose=1)
model_checkpoint_cb = model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='resnet50_untuned.h5',
    monitor='val_sparse_categorical_accuracy',
    save_best_only=True)
history = model.fit(x=train_set, validation_data=val_set, epochs=initial_epochs, verbose=1,
                    callbacks=[early_stopping_cb, reduce_lr, model_checkpoint_cb])

In [ ]:
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### Fine-tune model

In [ ]:
# inspect base model
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

In [ ]:
# unfreeze layers of the ResNet50 base model
base_model.trainable = True
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, 
              loss='sparse_categorical_crossentropy', 
              metrics=['sparse_categorical_accuracy'],
              )

In [ ]:
fine_epochs = 20
total_epochs = len(history.history['loss']) + fine_epochs
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1)
early_stopping_cb =  tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, verbose=1)
model_checkpoint_cb = model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='resnet50_finetuned.h5',
    monitor='val_sparse_categorical_accuracy',
    save_best_only=True)
history_fine = model.fit(x=train_set, validation_data=val_set, epochs=total_epochs, initial_epoch=history.epoch[-1]+1, verbose=1,
                        callbacks=[reduce_lr, early_stopping_cb, model_checkpoint_cb])

In [ ]:
acc += history_fine.history['sparse_categorical_accuracy']
val_acc += history_fine.history['val_sparse_categorical_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
history_fine.epoch